In [12]:
import sys
import pandas as pd
import numpy as np
import glob
import os
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import statistics as st
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.feature_selection import mutual_info_classif
import tensorflow as tf


qud = pd.read_csv(r"C:\Users\sophi\Dokumente\Studium\Wirtschaft\Bachelorarbeit\Datasets\QUD-dataset-master\QUD-dataset-master\QUD_app-1 (desktop).csv")
qud.info()
qud_Xy = qud.iloc[:, 3:8]
qud_X = qud.iloc[:, 3:7]
qud_y = qud.MicroMoments


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46930 entries, 0 to 46929
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   qdate             46930 non-null  object 
 1   time              46930 non-null  object 
 2   appID             46930 non-null  object 
 3   occupancy         46930 non-null  int64  
 4   PowerConsumption  46930 non-null  float64
 5   NormalizedPower   46930 non-null  float64
 6   QuantizedPower    46930 non-null  float64
 7   MicroMoments      46930 non-null  int64  
dtypes: float64(3), int64(2), object(3)
memory usage: 2.9+ MB


# DNN Model

In [13]:

def create_feature_column():
    feat_p = tf.feature_column.numeric_column('PowerConsumption')
    feat_o = tf.feature_column.numeric_column('occupancy')
    feat_pn = tf.feature_column.numeric_column('NormalizedPower')
    feat_pq = tf.feature_column.numeric_column('QuantizedPower')

    feature_column = [feat_p, feat_o,
                      feat_pn, feat_pq]

    return feature_column


feature_columns = create_feature_column()

DNN_model = tf.estimator.DNNClassifier(
    hidden_units=[10, 20, 20, 20, 20, 20, 20, 20, 20, 5], feature_columns=feature_columns,
    model_dir=None, n_classes=5, optimizer='Adam', activation_fn=tf.nn.relu, dropout=None)




INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\sophi\\AppData\\Local\\Temp\\tmpiv38cgcz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Accuracy

### 10-Fold Cross-Validation DNN

In [6]:

k = 10
kf = KFold(n_splits=k, random_state=1, shuffle=True)

acc_score = []

for train_index, test_index in kf.split(qud_X):
    X_train, X_test = qud_X.iloc[train_index, :], qud_X.iloc[test_index, :]
    y_train, y_test = qud_y[train_index], qud_y[test_index]

    input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(X_train, y_train,
                                                               num_epochs=1000, shuffle=True)
    predict_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, num_epochs=1, shuffle=False)

    DNN_model.train(input_fn=input_func, steps=500)
    predictions = list(DNN_model.predict(input_fn=predict_input_fn))
    prediction = [p["class_ids"][0] for p in predictions]
    # data = classification_report(Y_Test, prediction)

    acc = accuracy_score(y_test, prediction)
    acc_score.append(acc)

avg_acc_score = sum(acc_score) / k

print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.6149417, step = 0
INFO:tensorflow:global_step/sec: 345.324
INFO:tensorflow:loss = 1

INFO:tensorflow:global_step/sec: 372.129
INFO:tensorflow:loss = 0.0073954533, step = 1600 (0.278 sec)
INFO:tensorflow:global_step/sec: 525.87
INFO:tensorflow:loss = 0.13494134, step = 1700 (0.181 sec)
INFO:tensorflow:global_step/sec: 555.33
INFO:tensorflow:loss = 0.003413876, step = 1800 (0.180 sec)
INFO:tensorflow:global_step/sec: 523.908
INFO:tensorflow:loss = 0.24120863, step = 1900 (0.191 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 2000...
INFO:tensorflow:Saving checkpoints for 2000 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 2000...
INFO:tensorflow:Loss for final step: 0.10721895.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_

INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 4000...
INFO:tensorflow:Saving checkpoints for 4000 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 4000...
INFO:tensorflow:loss = 0.035540916, step = 4000
INFO:tensorflow:global_step/sec: 409.127
INFO:tensorflow:loss = 0.010209304, step = 4100 (0.244 sec)
INFO:tensorflow:global_step/sec: 555.975
INFO:tensorflow:loss = 0.022710148, step = 4200 (0.180 sec)
INFO:tensorflow:global_step/sec: 530.763
INFO:tensorflow:loss = 0.07530107, step = 4300 (0.189 sec)
INFO:tensorflow:global_step/sec: 550.117
INFO:tensorflow:loss = 0.20982295, step = 4400 (0.181 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 4500...
INFO:tensorflow:Saving 

### Stratified 10-Fold Cross Validation

In [7]:
k = 10
kf = StratifiedKFold(n_splits=k)

acc_score = []

for train_index, test_index in kf.split(qud_X, qud_y):
    X_train, X_test = qud_X.iloc[train_index, :], qud_X.iloc[test_index, :]
    y_train, y_test = qud_y[train_index], qud_y[test_index]

    input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(X_train, y_train,
                                                               num_epochs=1000, shuffle=True)
    predict_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, num_epochs=1, shuffle=False)

    DNN_model.train(input_fn=input_func, steps=500)
    predictions = list(DNN_model.predict(input_fn=predict_input_fn))
    prediction = [p["class_ids"][0] for p in predictions]
    # data = classification_report(Y_Test, prediction)

    acc = accuracy_score(y_test, prediction)
    acc_score.append(acc)

avg_acc_score = sum(acc_score) / k

print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))




INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.09143364, step = 5000
INFO:tensorflow:global_step/sec: 333.359
INFO:tensorflow:loss = 0.0020966623, step = 5100 (0.308 sec)
INFO:tensorflow:global_step/sec: 496.524
INFO:tensorflow:loss = 0.10210939, step = 5200 (0.194 sec)
INFO:tensorflow:global_step/sec: 555.616
INFO:tensorflow:loss = 0.015540426, step = 5300 (0.189 sec)
INFO:tensorflow:global_step/sec: 479.072
INFO:t

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-7500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 7500...
INFO:tensorflow:Saving checkpoints for 7500 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 7500...
INFO:tensorflow:loss = 0.00079293747, step = 7500
INFO:tensorflow:global_step/sec: 345.099
INFO:tensorflow:loss = 0.06406341, step = 7600 (0.290 sec)
INFO:tensorflow:global_step/sec: 499.819
INFO:tensorflow:loss = 0.07060774, step = 7700 (0.200 sec)
INFO:tensorflow:global_step/sec: 591.928
INFO:tensorflow:loss = 

INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
accuracy of each fold - [0.9965906669507778, 0.9872150010654166, 0.9942467504794374, 0.9906243341146388, 0.9889196675900277, 0.9942467504794374, 0.9870019177498401, 0.9899850841679096, 0.9976560835286598, 0.9897720008523333]
Avg accuracy : 0.9916258256978479


# F1 Score

### Stratified 10-Fold Cross Validation

In [8]:
from sklearn.metrics import f1_score


In [9]:
k = 10
kf = StratifiedKFold(n_splits=k)

f1score = []

for train_index, test_index in kf.split(qud_X, qud_y):
    X_train, X_test = qud_X.iloc[train_index, :], qud_X.iloc[test_index, :]
    y_train, y_test = qud_y[train_index], qud_y[test_index]

    input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(X_train, y_train,
                                                               num_epochs=1000, shuffle=True)
    predict_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, num_epochs=1, shuffle=False)

    DNN_model.train(input_fn=input_func, steps=500)
    predictions = list(DNN_model.predict(input_fn=predict_input_fn))
    prediction = [p["class_ids"][0] for p in predictions]
    # data = classification_report(Y_Test, prediction)

    f1 = f1_score(y_test, prediction, average='macro')
    f1score.append(f1)

avg_f1_score = sum(f1score) / k

print('F1 Score of each fold - {}'.format(f1score))
print('Avg F1 Score : {}'.format(avg_f1_score))



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...
INFO:tensorflow:Saving checkpoints for 10000 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...
INFO:tensorflow:loss = 0.00019586347, step = 10000
INFO:tensorflow:global_step/sec: 322.601
INFO:tensorflow:loss = 0.007370443, step = 10100 (0.310 sec)
INFO:tensorflow:global_step/sec: 474.397
INFO:tensorflow:loss = 0.018118815, step = 10200 (0.211 sec)
INFO:tensorflow:global_step/sec: 529.828
INFO:tensorflow:loss = 0.008704798, step = 10300 (0.189 sec)
INFO:tensorflow:global_step/sec: 550

INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-12500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-12500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 12500...
INFO:tensorflow:Saving checkpoints for 12500 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 12500...
INFO:tensorflow:loss = 0.6039424, step = 12500
INFO:tensorflow:global_step/sec: 344.791
INFO:tensorflow:loss = 0.070891045, step = 12600 (0.291 sec)
INFO:tensorflow:global_step/sec: 527.243
INFO:tensorflow

INFO:tensorflow:Loss for final step: 0.010394396.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
F1 Score of each fold - [0.9949116990516897, 0.9704031245186832, 0.9692602996778049, 0.9644512088261022, 0.9726134707178028, 0.9970565857490588, 0.9658998245685655, 0.9814032797683963, 0.9967677277748616, 0.9766255869169032]
Avg F1 Score : 0.9789392807569868


### 10-Fold-Cross-Validation

In [10]:
k = 10
kf = KFold(n_splits=k, random_state=1, shuffle=True)

f1score = []

for train_index, test_index in kf.split(qud_X):
    X_train, X_test = qud_X.iloc[train_index, :], qud_X.iloc[test_index, :]
    y_train, y_test = qud_y[train_index], qud_y[test_index]

    input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(X_train, y_train,
                                                               num_epochs=1000, shuffle=True)
    predict_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, num_epochs=1, shuffle=False)

    DNN_model.train(input_fn=input_func, steps=500)
    predictions = list(DNN_model.predict(input_fn=predict_input_fn))
    prediction = [p["class_ids"][0] for p in predictions]
    # data = classification_report(Y_Test, prediction)

    f1 = f1_score(y_test, prediction, average='macro')
    f1score.append(f1)

avg_f1_score = sum(f1score) / k

print('F1 Score of each fold - {}'.format(f1score))
print('Avg F1 Score : {}'.format(avg_f1_score))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 15000...
INFO:tensorflow:Saving checkpoints for 15000 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 15000...
INFO:tensorflow:loss = 0.004704829, step = 15000
INFO:tensorflow:global_step/sec: 349.641
INFO:tensorflow:loss = 0.030901713, step = 15100 (0.286 sec)
INFO:tensorflow:global_step/sec: 531.792
INFO:tensorflow:loss = 0.0051673, step = 15200 (0.188 sec)
INFO:tensorflow:global_step/sec: 537.489
INFO:tensorflow:loss = 0.06909634, step = 15300 (0.186 sec)
INFO:tensorflow:global_step/sec: 515.594


INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-17500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-17500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 17500...
INFO:tensorflow:Saving checkpoints for 17500 into C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 17500...
INFO:tensorflow:loss = 0.041623533, step = 17500
INFO:tensorflow:global_step/sec: 298.156
INFO:tensorflow:loss = 0.01368646, step = 17600 (0.336 sec)
INFO:tensorflow:global_step/sec: 554.082
INFO:tensorflo

INFO:tensorflow:Loss for final step: 0.005314879.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpnmoapys0\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
F1 Score of each fold - [0.9883992952860134, 0.9794936198998905, 0.9896318078806399, 0.9830097450068209, 0.9863847062684574, 0.9738928118367658, 0.9776795627568445, 0.9823921997035214, 0.9832666835402482, 0.9653672715035213]
Avg F1 Score : 0.9809517703682722


## Confusion Matrix


In [15]:
from sklearn.metrics import confusion_matrix
k = 10
kf = StratifiedKFold(n_splits=k)

Conf = []

for train_index, test_index in kf.split(qud_X, qud_y):
    X_train, X_test = qud_X.iloc[train_index, :], qud_X.iloc[test_index, :]
    y_train, y_test = qud_y[train_index], qud_y[test_index]

    input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(X_train, y_train,
                                                               num_epochs=1000, shuffle=True)
    predict_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, num_epochs=1, shuffle=False)

    DNN_model.train(input_fn=input_func, steps=500)
    predictions = list(DNN_model.predict(input_fn=predict_input_fn))
    prediction = [p["class_ids"][0] for p in predictions]
    # data = classification_report(Y_Test, prediction)

    cm = confusion_matrix(y_test, prediction)
    Conf.append(cm)

avg_confu = sum(Conf) / k

print('Confu: {}'.format(Conf))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpiv38cgcz\model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 500...
INFO:tensorflow:Saving checkpoints for 500 into C:\Users\sophi\AppData\Local\Temp\tmpiv38cgcz\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 500...
INFO:tensorflow:loss = 0.79921407, step = 500
INFO:tensorflow:global_step/sec: 310.877
INFO:tensorflow:loss = 0.5446723, step = 600 (0.323 sec)
INFO:tensorflow:global_step/sec: 399.466
INFO:tensorflow:loss = 0.81059414, step = 700 (0.249 sec)
INFO:tensorflow:global_step/sec: 427.428
INFO:tensorflow:loss = 0.17055996, step = 800 (0.235 sec)
INFO:tensorflow:global_step/sec: 474.015
INFO:tensorflow:lo

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpiv38cgcz\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 3000...
INFO:tensorflow:Saving checkpoints for 3000 into C:\Users\sophi\AppData\Local\Temp\tmpiv38cgcz\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 3000...
INFO:tensorflow:loss = 0.008380093, step = 3000
INFO:tensorflow:global_step/sec: 264.512
INFO:tensorflow:loss = 0.12318342, step = 3100 (0.379 sec)
INFO:tensorflow:global_step/sec: 391.875
INFO:tensorflow:loss = 0.020954888, step = 3200 (0.255 sec)
INFO:tensorflow:global_step/sec: 435.095
INFO:tensorflow:loss = 0

INFO:tensorflow:Restoring parameters from C:\Users\sophi\AppData\Local\Temp\tmpiv38cgcz\model.ckpt-5500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Avg Confu: [array([[1190,    0,    6,    0,   15],
       [   0,  157,    0,    0,    0],
       [   0,    0,  158,    0,    0],
       [   0,    0,    0,  393,    2],
       [   0,    0,    0,    0, 2772]], dtype=int64), array([[1189,    0,   20,    1,    1],
       [   0,  156,    0,    0,    0],
       [   0,    0,  157,    0,    1],
       [  47,    0,    0,  346,    3],
       [   0,    0,    0,    0, 2772]], dtype=int64), array([[1190,    0,   21,    0,    0],
       [   0,  156,    0,    0,    0],
       [   0,    0,  158,    0,    0],
       [   0,    0,    0,  396,    0],
       [   0,    0,    0,   55, 2717]], dtype=int64), array([[1197,    0,   13,    0,    0],
       [   0,  157,    0,    0,    0],
       [   0,    0,  157,    0,    1],
       [   0,    0,    0,  396,    0],
       [   5,  